In [94]:
import pandas as pd
from datetime import datetime, timedelta

# Set up location and time point

In [179]:
# some point in North Pacific Ocean
target_longitude = -170.2
target_latitude = 40.4
# how large should the region around the target location be for averaging values
region_padding_degrees = 0.0
# how many days back do we look for averaging values
days_back = 10
# average salinity/temp values in the region (yes/no)
average_location = False
# output dataframe (df) or xarray
output = 'xarray'

# current time
time_point = datetime.now()
# another point in time
#time_point = datetime(2023, 12, 25)

# Retrieve data from Copernicus API

In [177]:
import copernicusmarine

def format_data_points(current_datetime, days_back = 30):
    current_date_str = current_datetime.strftime("%Y-%m-%d")
    earlier_datetime = current_datetime - timedelta(days=days_back)
    earlier_date_str = earlier_datetime.strftime("%Y-%m-%d")
    return earlier_date_str, current_date_str

def copernicus_salinity_temp(target_longitude, region_padding_degrees, time_point, average_location = False, output = 'df'):
    start_time, end_time = format_data_points(time_point)
    
    salinity_ds = copernicusmarine.open_dataset(
        dataset_id = "cmems_mod_glo_phy-so_anfc_0.083deg_P1D-m",
        minimum_longitude = target_longitude-region_padding_degrees,
        maximum_longitude = target_longitude+region_padding_degrees,
        minimum_latitude = target_latitude-region_padding_degrees,
        maximum_latitude = target_latitude+region_padding_degrees,
        start_datetime = start_time,
        end_datetime = end_time,
        variables = ["sea_water_salinity"],
        # USER DATA ARE KEPT FOR HISTORY REASONS, CAN BE REPLACED WITH SOMETHING MEANINGFUL
        username='test',
        password='test'
    )
    
    temp_ds = copernicusmarine.open_dataset(
        dataset_id = "cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m",
        minimum_longitude = target_longitude-region_padding_degrees,
        maximum_longitude = target_longitude+region_padding_degrees,
        minimum_latitude = target_latitude-region_padding_degrees,
        maximum_latitude = target_latitude+region_padding_degrees,
        start_datetime = start_time,
        end_datetime = end_time,
        variables = ["sea_water_potential_temperature"],
        # USER DATA ARE KEPT FOR HISTORY REASONS, CAN BE REPLACED WITH SOMETHING MEANINGFUL
        username='test',
        password='test'
    )

    if output == 'df':
        salinity_ds = salinity_ds.to_dataframe().reset_index() 
        temp_ds = temp_ds.to_dataframe().reset_index()
        measurement_df = temp_ds.merge(salinity_ds, on = ['depth', 'latitude', 'longitude', 'time'])
        if average_location:
            measurement_df = measurement_df.drop(columns = ['time', 'latitude', 'longitude']).groupby(['depth']).mean().reset_index()
        else: 
            measurement_df = measurement_df.drop(columns = ['time']).groupby(['depth', 'latitude', 'longitude']).mean().reset_index()
        return measurement_df.dropna()
        
    else:
        salinity_ds['thetao'] = temp_ds['thetao']
        if average_location:
            salinity_ds = salinity_ds.mean(dim=['time', 'latitude', 'longitude'])
        else:
            salinity_ds = salinity_ds.mean(dim=['time'])

        return salinity_ds

In [182]:
measurement_ds = copernicus_salinity_temp(target_longitude, region_padding_degrees, time_point, average_location = average_location, output = output)
measurement_ds

INFO - 2024-05-16T07:56:47Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-05-16T07:56:47Z - Dataset part was not specified, the first one was selected: "default"
WARNING - 2024-05-16T07:56:47Z - The dataset cmems_mod_glo_phy-so_anfc_0.083deg_P1D-m, version '202211', will be retired on the 2024-07-26T00:00:00.000Z. After this date, it will no longer be available on the toolbox.
INFO - 2024-05-16T07:56:48Z - Service was not specified, the default one was selected: "arco-time-series"
INFO - 2024-05-16T07:56:50Z - Dataset version was not specified, the latest one was selected: "202211"
INFO - 2024-05-16T07:56:50Z - Dataset part was not specified, the first one was selected: "default"
WARNING - 2024-05-16T07:56:50Z - The dataset cmems_mod_glo_phy-thetao_anfc_0.083deg_P1D-m, version '202211', will be retired on the 2024-07-26T00:00:00.000Z. After this date, it will no longer be available on the toolbox.
INFO - 2024-05-16T07:56:51Z - Service was not sp

<xarray.Dataset> Size: 608B
Dimensions:    (depth: 50, latitude: 1, longitude: 1)
Coordinates:
  * depth      (depth) float32 200B 0.494 1.541 2.646 ... 5.275e+03 5.728e+03
  * latitude   (latitude) float32 4B 40.42
  * longitude  (longitude) float32 4B -170.2
Data variables:
    so         (depth, latitude, longitude) float32 200B 33.87 33.87 ... nan
    thetao     (depth, latitude, longitude) float32 200B 11.97 11.95 ... nan

# Compute sound speed

In [197]:
# Compute the speed of sound using Mackenzie’s formula
def compute_sound_speed_df(T, S, D):
    T_squared = T**2
    T_cubed = T**3
    D_squared = D**2
    D_cubed = D**3
    C = (1448.96 + 
         4.591*T - 
         5.304e-2*T_squared + 
         2.374e-4*T_cubed + 
         1.340*(S-35) + 
         1.630e-2*D + 
         1.675e-7*D_squared - 
         1.025e-2*T*(S-35) - 
         7.139e-13*T*D_cubed)
    return C

def compute_sound_speed_xarray(thetao, so, depth):
    return 1448.96 + 4.591 * thetao - 0.05304 * (thetao ** 2) + 2.374e-4 * (thetao ** 3) + 1.34 * (so - 35) + 0.0163 * depth

In [199]:
def compute_sound_speed(measurement_ds, output='dataset'):
    if output == 'df':
        measurement_df = measurement_ds.to_dataframe().reset_index()
        measurement_df['SOUND_SPEED'] = measurement_df.apply(
            lambda row: compute_sound_speed_xarray(row['thetao'], row['so'], row['depth']),
            axis=1
        )
        return measurement_df
    
    else:
        # Ensure that depth is aligned properly with thetao and so
        thetao, so, depth = xr.broadcast(measurement_ds["thetao"], measurement_ds["so"], measurement_ds["depth"])
        
        # Apply the compute_sound_speed_xarray function
        measurement_ds["SOUND_SPEED"] = xr.apply_ufunc(
            compute_sound_speed_xarray,
            thetao,
            so,
            depth,
            input_core_dims=[[], [], []],
            vectorize=True
        )
    
    return measurement_ds

measurement_ds = compute_sound_speed(measurement_ds, output = output)
measurement_ds

<xarray.Dataset> Size: 2kB
Dimensions:      (depth: 50, latitude: 1, longitude: 1)
Coordinates:
  * depth        (depth) float64 400B 0.0 20.41 40.82 ... 959.2 979.6 1e+03
  * latitude     (latitude) float64 8B 40.42
  * longitude    (longitude) float64 8B -170.2
Data variables:
    so           (depth, latitude, longitude) float64 400B 0.4281 ... 0.1237
    thetao       (depth, latitude, longitude) float64 400B 0.9601 ... 0.4981
    SOUND_SPEED  (depth, latitude, longitude) float64 400B 1.407e+03 ... 1.42...